## Import and Functions

In [4]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

## Read CSV

In [6]:
articles = pd.read_csv('Data/out_content.zip', index_col='article_id')
articles.head()

,product_group_name_Accessories,product_group_name_Bags,product_group_name_Cosmetic,product_group_name_Fun,product_group_name_Furniture,product_group_name_Garment Full body,product_group_name_Garment Lower body,product_group_name_Garment Upper body,product_group_name_Garment and Shoe care,product_group_name_Interior textile,...,garment_group_name_Shorts,garment_group_name_Skirts,garment_group_name_Socks and Tights,garment_group_name_Special Offers,garment_group_name_Swimwear,garment_group_name_Trousers,garment_group_name_Trousers Denim,"garment_group_name_Under-, Nightwear",garment_group_name_Unknown,garment_group_name_Woven/Jersey/Knitted mix Baby
article_id,,,,,,,,,,,,,,,,,,,,,
108775015,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
108775044,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
108775051,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
110065001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
110065002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
